In [1]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [2]:
panorama = cv2.imread(r"images\1330_1123_878_631_2.png",cv2.COLOR_BGR2GRAY)
image = cv2.imread(r"images\970.png",cv2.COLOR_BGR2GRAY)
img2 = cv2.imread("image_test/a.png",cv2.COLOR_BGR2GRAY)

In [3]:
# 使用 SIFT 特征检测器和匹配器
sift = cv2.SIFT_create()

# 在全景图和图像中检测特征点和计算特征描述符
keypoints_panorama, descriptors_panorama = sift.detectAndCompute(panorama, None)
keypoints_image, descriptors_image = sift.detectAndCompute(image, None)

# 使用匹配器来匹配特征点
matcher = cv2.BFMatcher()
matches = matcher.knnMatch(descriptors_panorama, descriptors_image, k=2)

# 应用 Lowe's 比率测试来筛选匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.5 * n.distance:
        good_matches.append(m)

# 提取匹配的关键点
matched_keypoints_panorama = [keypoints_panorama[match.queryIdx] for match in good_matches]
matched_keypoints_image = [keypoints_image[match.trainIdx] for match in good_matches]

In [4]:
# 获取匹配关键点的坐标
panorama_pts = np.float32([kp.pt for kp in matched_keypoints_panorama])
image_pts = np.float32([kp.pt for kp in matched_keypoints_image])
# 1 is feature 2 is image
x1, y1, w1, h1 = cv2.boundingRect(image_pts)
h2, w2 = image.shape[:2] 
x2 = 0
y2 = 0

lr = (x1-x2)/w1
ur = (y1-y2)/h1
der = h2/h1
rer = w2/w1
#3 is feature pano  4 is real area
x3, y3, w3, h3 = cv2.boundingRect(panorama_pts)
x4 = x3-w3*lr
y4 = y3-h3*ur
w4 = w3*rer
h4 = h3*der

# 创建一个空白 mask 图像
mask = np.zeros_like(panorama, dtype=np.uint8)
mask = mask.astype(np.uint8)  # 确保 mask 是 uint8 类型的 NumPy 数组
# 在 mask 区域内填充白色
p4 = np.array([[x4, y4], [x4 + w4, y4], [x4 + w4, y4 + h4], [x4, y4 + h4]],dtype=np.int32)

cv2.fillPoly(mask, [p4], (255,255,255))
# 将 NumPy 数组转换为 Pillow 图像
mask_pillow = Image.fromarray(mask)
# print(x1, y1, w1, h1)
# print(x2,y2,w2,h2)
# print(x3, y3, w3, h3)
# print(x4,y4,w4,h4)
# print(panorama.shape)
# print(image.shape)
# print(img2.shape)
# 显示 Pillow 图像（可选）
mask_pillow.show()

In [5]:
non_zero_indices = np.where(mask > 0)
masked_image1 = np.zeros_like(panorama)
# mask1==0的位置保留image1值
masked_image1[mask != 0] = panorama[mask != 0]
cv2.imshow("Result", masked_image1)
pillow_image = Image.fromarray(cv2.cvtColor(masked_image1, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [13]:
mask_part = panorama[p4]
a = Image.fromarray(mask_part)
a.show()

IndexError: index 2728 is out of bounds for axis 0 with size 433

In [ ]:
import cv2      
import numpy as np
from PIL import Image      
# 创建一个黑色背景的图像      
img = np.zeros((500, 500, 3), dtype=np.uint8)      
# 在图像上绘制一个红色的三角形      
pts = np.array([[100, 100], [300, 100], [200, 300]])    
cv2.fillPoly(img, [pts], (0, 0, 255))      
# 显示图像   
pillow_image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [ ]:
mask_part = panorama[pts]
a = Image.fromarray(mask_part)
a.show()

TypeError: Cannot handle this data type: (1, 1, 1338, 3), |u1